# Notebook 6 – Geometric Fingerprints of Particles

**Level:** Intermediate  
**Topics:** Geometric diagnostics, Fisher metric, Berry curvature, KK metric, ξ, Reeb flow

> **Provenance:** This notebook is based on:
> - `examples/pocket_u_geometry_showcase.py`
> - `examples/pocket_u_mass_table.py`

In [ ]:
# %pip install light-theory-realm pandas matplotlib

import pandas as pd
import matplotlib.pyplot as plt

from light_theory_realm import get_particle_profile

try:
    from light_theory_realm.pocket_u_lite.sm_table import ALL_PARTICLES
except ImportError:
    ALL_PARTICLES = ["e", "mu", "tau", "u", "d", "s", "c", "b", "t"]

plt.rcParams["figure.figsize"] = (7, 5)

## 1. Building a geometry table

> Note: This notebook includes a small robustness patch for `compute_reeb_diagnostics` (safe to re-run; idempotent).


In [ ]:
# Monkeypatch for robustness (same as Notebook 1 & 4)
import light_theory_realm.pocket_u_lite.geometry as geom

# Only save the original once, even if you re-run this cell
if not hasattr(geom, "_orig_compute_reeb_diagnostics"):
    geom._orig_compute_reeb_diagnostics = geom.compute_reeb_diagnostics

def compute_reeb_diagnostics_fixed(primes, *args, **kwargs):
    """
    Wrapper around compute_reeb_diagnostics that ensures `primes` is a tuple,
    so expressions like `primes + (primes[0],)` work even if `primes` was a list.
    """
    primes = tuple(primes)  # list -> tuple (tuple -> tuple is also fine)
    return geom._orig_compute_reeb_diagnostics(primes, *args, **kwargs)

# Monkeypatch the module-level function used by get_particle_geometry
geom.compute_reeb_diagnostics = compute_reeb_diagnostics_fixed
print("Patched pocket_u_lite.geometry.compute_reeb_diagnostics (idempotent).")

profiles = []
for name in ALL_PARTICLES:
    p = get_particle_profile(name)
    profiles.append(p)

df = pd.DataFrame(profiles)
df

## 2. Mass vs Fisher trace

In [ ]:
df_mass = df.sort_values("m_phys_MeV")

plt.figure()
plt.scatter(df_mass["m_phys_MeV"], df_mass["fisher_trace"])
for _, row in df_mass.iterrows():
    plt.text(row["m_phys_MeV"], row["fisher_trace"], row["name"], fontsize=8)

plt.xscale("log")
plt.xlabel("Predicted mass m_phys_MeV [MeV] (log)")
plt.ylabel("fisher_trace")
plt.title("Fisher trace vs mass")
plt.grid(True, which="both", linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()

## 3. Mass vs Berry norm

In [ ]:
plt.figure()
plt.scatter(df_mass["m_phys_MeV"], df_mass["berry_norm"])
for _, row in df_mass.iterrows():
    plt.text(row["m_phys_MeV"], row["berry_norm"], row["name"], fontsize=8)

plt.xscale("log")
plt.xlabel("Predicted mass m_phys_MeV [MeV] (log)")
plt.ylabel("berry_norm")
plt.title("Berry norm vs mass")
plt.grid(True, which="both", linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()

## 4. ξ vs screening (vacuum vs matter)

In [ ]:
plt.figure()
plt.scatter(df["screening_pct"], df["xi"])
for _, row in df.iterrows():
    plt.text(row["screening_pct"], row["xi"], row["name"], fontsize=8)

plt.xlabel("Screening [%]")
plt.ylabel("ξ (Reeb resonance density)")
plt.title("ξ vs screening across Standard Model particles")
plt.grid(True, linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()

## 5. Electron vs tau: geometric fingerprint

In [ ]:
def show_fingerprint(name):
    row = df[df["name"] == name].iloc[0]
    print(f"=== {name} ===")
    for key in ["m_phys_MeV", "pdg_mass_MeV",
                "fisher_trace", "berry_norm", "xi",
                "R_norm", "screening_pct"]:
        if key in row:
            print(f"{key:16s}: {row[key]}")

show_fingerprint("e")
print()
show_fingerprint("tau")


## 6. Insights

Some questions to think about:

- Do **heavier particles** tend to have larger Fisher or Berry norms?
- How does **screening** correlate with ξ?
- How do the **electron** and **tau** differ geometrically, beyond just their masses?

---

**Next steps:**

- Try plotting **R_norm vs mass** or **G_trace vs mass**.
- Compare quarks vs leptons via color-coding or separate plots.
- For deeper prime / Reeb structure, continue later to:
  - **Notebook 7 – Prime Plaquettes & Topological Mass**
  - **Notebook 8 – Dark Energy Toy Model**